# Distilroberta-finetuned-financial-news-sentiment-analysis

In [78]:
#!pip install ktrain
#!pip install sentence_transformers

In [190]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text

#from sentence_transformers import SentenceTransformer

data_set = pd.read_csv(r"C:\Users\volpe\Documents\MARCO\0.master\0.Corsi\Machine_learning\Carman_google_drive\TextMining\data_0905_semicolon.csv",sep=";")

In [191]:
data_set.head()

,Unnamed: 0,Ticker,Titles,Date,Var_prc,Target
0,0,MMM,<title>Here is What to Know Beyond Why 3M Comp...,0,0.008478,0.0
1,69,MMM,<title>What's Next For 3M Stock After A 15% Fa...,0,0.008478,0.0
2,68,MMM,<title>MMM Stock Quote Price and Forecast - CN...,0,0.008478,0.0
3,67,MMM,<title>3M Announces Upcoming Investor Event - ...,0,0.008478,0.0
4,66,MMM,<title>Open Door: Claire Pentecost + Beth Jaco...,0,0.008478,0.0


In [192]:
# we want a input dataframe where there is only a row for each ticker, 10 columns with the 10 titles, and the target
number_articles = 10
col =["Ticker"] + [ "Article " + str(i+1)  for i in range(number_articles)] + ["Target"]
input_df = pd.DataFrame(columns=col)

for ticker in data_set["Ticker"].unique():
    #extract the rows corresponfing to the ticker
    ticker_df = data_set[ data_set["Ticker"] ==  ticker ]
    #create the new row to append to the dataset
    articles_list = ticker_df["Titles"].to_list()
    ticker_target = ticker_df["Target"].iloc[0]
    new_row = [ticker] + articles_list + [ticker_target] 
    #append the new row to the dataset
    input_df = input_df.append(pd.Series(new_row, index=col),ignore_index=True)
input_df = input_df.dropna()
print(input_df.shape)
input_df.iloc[60:63,:]

(498, 12)


,Ticker,Article 1,Article 2,Article 3,Article 4,Article 5,Article 6,Article 7,Article 8,Article 9,Article 10,Target
60,BAX,<title>Biden is tweaking an asylum rule to spe...,<title>Baxter's (BAX) Latest FDA Approval to B...,<title>Mayor Sarno Joined with Bax and Nagle t...,"<title>Based On Its ROE, Is Baxter Internation...",<title>(01/02/24) Top Picks 2024: Baxter Inter...,"<title>Baxter (BAX) Beats on Q3 Earnings, Dive...",<title>Is Baxter International (BAX) Too Good ...,<title>Baxter International Inc.'s (NYSE:BAX) ...,<title>Author Correction: Bax Inhibitor-1 regu...,<title>Estimating The Fair Value Of Baxter Int...,-1.0
61,BDX,"<title>When the Price of (BDX) Talks, People L...","<title>Becton, Dickinson and Company (NYSE:BDX...",<title>Beyond the Balance Sheet: What SWOT Rev...,<title>Becton Dickinson (BDX) Surpasses Q3 Ear...,<title>Breaking: BDx Indonesia to acquire Indo...,<title>BD Increases Access to Cutting-Edge Ima...,<title>Investors Still Waiting For A Pull Back...,<title>BDx acquires data center freehold in Si...,<title>BDX Droids Coming to Star Wars Galaxy's...,<title>Why Becton Dickinson (BDX) is a Top Val...,-1.0
63,BBY,<title>Best Buy (NYSE:BBY) PT Lowered to $67.0...,<title>Best Buy (BBY) Up 11.1% Since Last Earn...,<title>Best Buy Reports Q4 FY24 Results - Best...,<title>Decoding Best Buy Co Inc (BBY): A Strat...,<title>Should You Think About Buying Best Buy ...,<title>This Insider Has Just Sold Shares In Be...,<title>Best Buy Geek Squad employees report ma...,<title>Best Buy Recalls Insignia® Air Fryers a...,"<title>Down 15% This Year, Where Is Best Buy S...",<title>Dividend Investors: Don't Be Too Quick ...,0.0


In [193]:
# Cleaning the columns and creating the input for the multilingual bert
for i in range(1, 11):
    column_name = f'Article {i}'
    input_df[column_name] = input_df[column_name].str.replace('<title>', '').str.replace('</title>', '')

input_df["Concatenated Titles"] = input_df.iloc[:, 1:11].apply(lambda row: ' '.join(row), axis=1)
input_df.dropna(axis = 0)
# Print the resulting DataFrame
input_df.iloc[50:53,:]

,Ticker,Article 1,Article 2,Article 3,Article 4,Article 5,Article 6,Article 7,Article 8,Article 9,Article 10,Target,Concatenated Titles
50,ADP,Ep. 539 - Analyzing the New Post-Draft Startup...,ADP Increases Cash Dividend; Marks 49th Consec...,Why Is Software Megacap ADP's Stock Trading Hi...,The Biggest ADP Fallers: How long Gerrit Cole ...,ADP Ventures | Creating a better world of work...,"Private payrolls rose by 140,000 in February, ...",ADP Reimagines the Human Capital Management Cl...,ADP: Employers Added 103000 Workers in Novembe...,ADP Declares Regular Quarterly Dividend - PR N...,ADP vs Paychex: Which Payroll Software is Bett...,0.0,Ep. 539 - Analyzing the New Post-Draft Startup...
51,AZO,(AZO) Investment Report - Stock Traders Daily,Light-tuned selective photosynthesis of azo- a...,Optimal of black anti-static AZO/Al2O3-SiO2-Vx...,"New azo-azomethine derivatives: Synthesis, cha...",NLO azo compounds with sulfonamide groups: A t...,Self-organized patterning on azo molecular gla...,"Synthesis, spectroscopic characterization and ...",Protocol for photoelectrocatalytic synthesis o...,Easily accessible and tunable porous organic p...,"Reversal in solvatochromism, photochromism and...",0.0,(AZO) Investment Report - Stock Traders Daily ...
52,AVB,Decoding AvalonBay Communities Inc (AVB): A St...,'Massive arguments' - AVB reveals Tottenham re...,Van der Vaart calls AVB 'an idiot' and explain...,10 Ways to Use AVB ('Already Vaped Bud') Marij...,Ruben Amorim to Liverpool: Will he become a Mo...,AVIATION LOGISTICS SUPPORT IN THE INDO-PACIFIC...,6 Easy Ways for Getting High Again with Your A...,‘This is really uncomfortable’ – John Terry an...,InfoComm 2023: DiGiCo Introduces Milan-Certifi...,Analysts See Upside On These 3 REITs - AvalonB...,0.0,Decoding AvalonBay Communities Inc (AVB): A St...


In [194]:
from sklearn.model_selection import train_test_split

X = input_df.filter(["Concatenated Titles"],axis=1)
Y = input_df.filter(["Target"],axis=1)
#print(Y)

x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
y_train = y_train + 1          # classes need to start from 0
y_test = y_test + 1

### Here starts model training

In [195]:
model_name = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
transformer = text.Transformer(model_name, maxlen=512, class_names= np.array([0,1,2]))

In [196]:
processed_train = transformer.preprocess_train(list(x_train["Concatenated Titles"]), list(y_train["Target"]))
processed_test = transformer.preprocess_test(list(x_test["Concatenated Titles"]), list(y_test["Target"]))

preprocessing train...
language: en
train sequence lengths:
	mean : 137
	95percentile : 160
	99percentile : 174


c:\Users\volpe\anaconda3\envs\kt\lib\site-packages\ktrain\utils.py:769: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  "class_names implies classification but targets array contains float(s) instead of integers or strings"


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 137
	95percentile : 164
	99percentile : 169


In [197]:
model = transformer.get_classifier()
batch_size = 5                        #small training dataset
learner = ktrain.get_learner(model, train_data=processed_train, val_data=processed_test, batch_size=batch_size)

In [198]:
print(processed_train.y)

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [199]:
learner.lr_find(show_plot=True,  max_epochs=1)

simulating training for different learning rates... this may take a few moments...
11/79 [===>..........................] - ETA: 8:00 - loss: 3.9224 - accuracy: 0.3636

In [ ]:
learner.fit_onecycle(0.5e-1, 2)



begin training using onecycle policy with max lr of 0.05...
Epoch 1/2
80/80 [==============================] - 518s 6s/step - loss: 3.8166 - accuracy: 0.5603 - val_loss: 1.9656 - val_accuracy: 0.1800
Epoch 2/2
80/80 [==============================] - 565s 7s/step - loss: 6.6673 - accuracy: 0.5402 - val_loss: 4.6888 - val_accuracy: 0.5900


In [ ]:
learner.validate(class_names=transformer.get_classes())

4/4 [==============================] - 27s 6s/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.59      1.00      0.74        59
           2       0.00      0.00      0.00        18

    accuracy                           0.59       100
   macro avg       0.20      0.33      0.25       100
weighted avg       0.35      0.59      0.44       100



c:\Users\volpe\anaconda3\envs\kt\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\volpe\anaconda3\envs\kt\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\volpe\anaconda3\envs\kt\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[ 0, 23,  0],
       [ 0, 59,  0],
       [ 0, 18,  0]], dtype=int64)

In [ ]:
'''
# let's see some predictions
'''
predictor = ktrain.get_predictor(learner.model, preproc=transformer)
import random as rand
i = rand.randint(0,100)

print(input_df["Ticker"].iloc[i],"\n",\
      "Prediction: \n",\
      predictor.predict(x_test["Concatenated Titles"].iloc[i]),\
      "\nTarget: \n",\
      y_train["Target"].iloc[i])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

It always predicts class 1... not really good

### Let's try embedding + NN

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
m = SentenceTransformer("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

No sentence-transformers model found with name C:\Users\volpe/.cache\torch\sentence_transformers\mrm8488_distilroberta-finetuned-financial-news-sentiment-analysis. Creating a new one with MEAN pooling.


In [ ]:
#len(m.encode("I love transformers!"))     #768 dimensional embedding
c = 768                   # dim embedding
r = input_df.shape[0]    # dim dataset  

X = np.zeros((r,c))
for idx in range(r):
  sentence_embeddings = m.encode(input_df.iloc[idx,:].loc["Concatenated Titles"])
  X[idx,:] = sentence_embeddings

In [ ]:
input_df.iloc[497,:].loc["Concatenated Titles"]

"Smoothie King names Yum Brands veteran Gavin Felder chief financial officer - Nation's Restaurant News Yum China Holdings, Inc. (NYSE:YUMC) Q1 2024 Earnings Call Transcript - Yahoo Finance Yum Brands Doubles Tech Spending, Expands Generative AI Use - PYMNTS.com Compared to Estimates, Yum China (YUMC) Q1 Earnings: A Look at Key Metrics - Yahoo Finance Yum! Brands, Inc. (NYSE:YUM) Q4 2023 Earnings Call Transcript - Yahoo Finance Yum China (YUMC) Q4 2023 Earnings Call Transcript - The Motley Fool Yum Brands (YUM) Reports Next Week: Wall Street Expects Earnings Growth - Yahoo Finance Pizza Hut Announces Carl Loredo President of Pizza Hut U.S. - Business Wire Yum China (NYSE:YUMC) Misses Q1 Sales Targets - Yahoo Finance Yum! Brands (YUM) Reports Earnings Tomorrow: What To Expect - Yahoo Finance"

In [ ]:
t_size = 0.2

x_train, x_test, y_train, y_test = train_test_split( X, np.array(input_df["Target"]), test_size=t_size, random_state=42)


In [ ]:
y_train = y_train + 1
y_test = y_test + 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, RMSprop, SGD, Adagrad

In [ ]:
#to train the NN we need to convert integer output to binary array
num_classes = 3

y_train_dummy = np.zeros((len(y_train), num_classes))
for i in range(0,len(y_train)):
    class_idx = round(y_train[i])  
    y_train_dummy[i, class_idx] = 1


y_test_dummy = np.zeros((len(y_test), num_classes))
for i in range(0,len(y_test)):
    y_test_dummy[i,round(y_test[i])] = 1

In [ ]:
model=Sequential()
model.add(Dense(units=3000, activation='linear', input_dim=c))
model.add(Dense(units=500, activation='tanh'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=3, activation='sigmoid'))    #output layer
model.compile(loss='mse',
              optimizer=Adam(lr=0.0001,beta_1=0.9,beta_2=0.999))

model.fit(x_train,y_train,epochs=200,batch_size=10)  #10, 5 


pred_train=model.predict(x_train)
pred_test=model.predict(x_test)

Epoch 1/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2967
Epoch 2/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2939
Epoch 3/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2940
Epoch 4/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2937
Epoch 5/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2938
Epoch 6/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2934
Epoch 7/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2934
Epoch 8/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2940
Epoch 9/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2938
Epoch 10/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2935
Epoch 11/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2934
Epoch 12/200
40/40 [==============================] - 1s 15ms/step - loss: 0.2925
Epoch 13/200
40/40 [=====

In [ ]:
'''
model=Sequential()
model.add(Dense(units=c, activation='relu', input_dim=c))
model.add(Dense(units=900, activation='tanh'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=3, activation='sigmoid'))    #output layer
model.compile(loss='mse',
              optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.999))

model.fit(x_train,y_train,epochs=200,batch_size=6)  


pred_train=model.predict(x_train)
pred_test=model.predict(x_test)

Missclasification_train:  0.29396984924623115
Missclassification_test:  0.41

predicts ONLY class 1...
'''

"\nmodel=Sequential()\nmodel.add(Dense(units=c, activation='relu', input_dim=c))\nmodel.add(Dense(units=900, activation='tanh'))\nmodel.add(Dense(units=200, activation='relu'))\nmodel.add(Dense(units=3, activation='sigmoid'))    #output layer\nmodel.compile(loss='mse',\n              optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.999))\n\nmodel.fit(x_train,y_train,epochs=200,batch_size=6)  \n\n\npred_train=model.predict(x_train)\npred_test=model.predict(x_test)\n\nMissclasification_train:  0.29396984924623115\nMissclassification_test:  0.41\n\npredicts ONLY class 1...\n"

In [ ]:
'''
model=Sequential()
model.add(Dense(units=c, activation='linear', input_dim=c))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=3, activation='sigmoid'))    #output layer
model.compile(loss='mse',
              optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.999))

model.fit(x_train,y_train,epochs=200,batch_size=6) 


pred_train=model.predict(x_train)
pred_test=model.predict(x_test)
Missclasification_train:  0.7412060301507538
Missclassification_test:  0.7
'''

"\nmodel=Sequential()\nmodel.add(Dense(units=c, activation='linear', input_dim=c))\nmodel.add(Dense(units=200, activation='relu'))\nmodel.add(Dense(units=3, activation='sigmoid'))    #output layer\nmodel.compile(loss='mse',\n              optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.999))\n\nmodel.fit(x_train,y_train,epochs=200,batch_size=6) \n\n\npred_train=model.predict(x_train)\npred_test=model.predict(x_test)\nMissclasification_train:  0.7412060301507538\nMissclassification_test:  0.7\n"

In [ ]:
pred_test_2 = np.zeros(pred_test.shape)
for i in range(len(pred_test)):
  pred_test_2[i, pred_test[i,:].argmax()  ] = 1

pred_train_2 = np.zeros(pred_train.shape)
for i in range(len(pred_train)):
  pred_train_2[i, pred_train[i,:].argmax()  ] = 1

In [ ]:
#print(pred_train.shape,"--",x_train.shape)
#print(pred_test.shape,"--",x_test.shape)

print(pred_train[23,:],"--",y_train_dummy[0,:])
print(  )
print("Prediction","--","  Target")
for i in range(180,200):
  print(pred_train_2[i,:],"--",y_train_dummy[i,:])

[0.999998   0.99999815 0.99999744] -- [0. 1. 0.]

Prediction --   Target
[0. 0. 1.] -- [0. 1. 0.]
[0. 1. 0.] -- [0. 1. 0.]
[0. 1. 0.] -- [0. 1. 0.]
[0. 0. 1.] -- [1. 0. 0.]
[0. 0. 1.] -- [0. 0. 1.]
[0. 0. 1.] -- [0. 1. 0.]
[1. 0. 0.] -- [1. 0. 0.]
[0. 0. 1.] -- [1. 0. 0.]
[0. 1. 0.] -- [1. 0. 0.]
[0. 1. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[0. 0. 1.] -- [1. 0. 0.]
[1. 0. 0.] -- [1. 0. 0.]
[0. 1. 0.] -- [0. 1. 0.]
[0. 0. 1.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[0. 1. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]


It predicts different classes!!!

In [ ]:
MSE_train = np.sum(  (pred_train-y_train_dummy)**2)   /pred_train.shape[0]
MSE_test = np.sum(  (pred_test-y_test_dummy)**2)   /pred_test.shape[0]

print("MSE_train: ",MSE_train)
print("MSE_test: ",MSE_test)


MSE_train = np.sum(  (pred_train_2-y_train_dummy)**2)   /pred_train.shape[0]
MSE_test = np.sum(  (pred_test_2-y_test_dummy)**2)   /pred_test.shape[0]

print("MSE_train: ",MSE_train)
print("MSE_test: ",MSE_test)

MSE_train:  1.8503334895666848
MSE_test:  1.8050412938987537
MSE_train:  1.3668341708542713
MSE_test:  1.36


In [ ]:
miss_train = np.sum(  abs(pred_train_2-y_train_dummy)/2)   /pred_train.shape[0]
miss_test = np.sum(  abs(pred_test_2-y_test_dummy)/2)   /pred_test.shape[0]

print("Missclasification_train: ",miss_train)
print("Missclassification_test: ",miss_test)

Missclasification_train:  0.6834170854271356
Missclassification_test:  0.68


In [ ]:
'''
model=Sequential()
model.add(Dense(units=c, activation='linear', input_dim=c))
model.add(Dense(units=800, activation='relu'))
model.add(Dense(units=3, activation='sigmoid'))    #output layer
model.compile(loss='mse',
              optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.999))

model.fit(x_train,y_train,epochs=10000,batch_size=6) 


Non-trivial predictions!!!!!!!!!!
'''

"\nmodel=Sequential()\nmodel.add(Dense(units=c, activation='linear', input_dim=c))\nmodel.add(Dense(units=800, activation='relu'))\nmodel.add(Dense(units=3, activation='sigmoid'))    #output layer\nmodel.compile(loss='mse',\n              optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.999))\n\nmodel.fit(x_train,y_train,epochs=10000,batch_size=6) \n\n\nNon-trivial predictions!!!!!!!!!!\n"

In [ ]:
'''
model=Sequential()
model.add(Dense(units=900, activation='relu', input_dim=c))
model.add(Dense(units=300, activation='tanh'))
model.add(Dense(units=3, activation='sigmoid'))    #output layer
model.compile(loss='mse',
              optimizer=Adam(lr=0.0001,beta_1=0.9,beta_2=0.999))

model.fit(x_train,y_train,epochs=200,batch_size=10)  #10, 5 


pred_train=model.predict(x_train)
pred_test=model.predict(x_test)

Missclasification_train:  0.5753768844221105
Missclassification_test:  0.53
'''

"\nmodel=Sequential()\nmodel.add(Dense(units=900, activation='relu', input_dim=c))\nmodel.add(Dense(units=300, activation='tanh'))\nmodel.add(Dense(units=3, activation='sigmoid'))    #output layer\nmodel.compile(loss='mse',\n              optimizer=Adam(lr=0.0001,beta_1=0.9,beta_2=0.999))\n\nmodel.fit(x_train,y_train,epochs=200,batch_size=10)  #10, 5 \n\n\npred_train=model.predict(x_train)\npred_test=model.predict(x_test)\n\nMissclasification_train:  0.5753768844221105\nMissclassification_test:  0.53\n"